### Задание 1

##### Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования. Алгоритмы:

#### Кнута-Морриса-Пратта

In [ ]:
def search_algo_1(string, substr):
    substr = list(substr)
    shifts = [0] * (len(substr))
    pos1 = 0
    pos2 = 1
    while pos1 < len(substr) - 1 and pos2 < len(substr):
        if substr[pos2] != substr[pos1]:
            if pos1 == 0:
                shifts[pos2] = 0
                pos2 += 1
            else:
                pos1 = shifts[pos1 - 1]
        else:
            shifts[pos2] = pos1 + 1
            pos1 += 1
            pos2 += 1

    start_position = 0
    match_length = 0
    current_pos = 0
    while current_pos < len(string) and match_length < len(substr):
        if string[current_pos] == substr[match_length]:
            match_length += 1
            current_pos += 1
        else:
            if match_length == 0:
                current_pos += 1
                start_position += 1
            else:
                match_length = shifts[match_length - 1]
                start_position += current_pos - start_position
    if match_length == len(substr):
        return start_position
    else:
        return -1

#### Упрощенный Бойера-Мура

In [ ]:
def offset_calc(substr):
    offset_table = [len(substr) for i in range(1105)]
    new_p = substr[::-1]
    for i in range(len(new_p)):
        if offset_table[ord(new_p[i])] != len(new_p):
            continue
        else:
            offset_table[ord(new_p[i])] = i
    return offset_table


def search_algo_2(string, substr):
    offset_table = offset_calc(substr)
    len_substr = i = j = k = len(substr)
    while i <= len(string) and j > 0:
        if string[k - 1] == substr[j - 1]:
            k -= 1
            j -= 1
        else:
            i += offset_table[ord(string[k - 1])]
            j = len_substr
            k = i
    if j <= 0:
        return k
    return None

### Задание 2 "Пятнашки"

In [5]:
import queue
import time

# Инверсия - когда больший элемент стоит до меньшего элемента
#Считаем для всех костяшек
def count_inversions(array):
    inversions = 0
    size = len(array)
    for i in range(0, size-1):
        for j in range(i+1, size):
            if array[i] > 0 and array[j] > 0:
                if array[i] > array[j]:
                    inversions += 1
    return inversions

# Для определения решаемы ли костяшки
def is_solvable(array, m=4, n=4):
    inversions = count_inversions(array)
    size = len(array)
    # В случае чётного количества костяшек смотрим, на какой строке снизу
    #находится пустое место (последняя строка - нечётное, предпоследняя - чётное и т.д.
    #для решаемости необходима разная чётность кол-ва инверсий и строки пустого места (чёт/нечёт)
    if n*m%2==0:
        for i in range(0, size):
            if array[i] == 0:
                if ((m-i//n)%2 == 1):
                    return(inversions%2 == 0)
                else:
                    return(inversions%2 != 0)
        return False
    # В случае нечётного количества костяшек нужно, чтобы количество инверсий было чётно
    if inversions%2==0:
        return True
    return False
    
# Класс состояния поля пятнашек
class BoardState:
    array = [] # Для хранения расположение костяшек
    n = 4 # Размерность поля
    m = 4
    pos0 = 0 # Для хранения позиции пустого места
    moves = [] # Для хранения необходимых перемещения для его достижения
    
    def __init__(self, array, n=4, m=4, pos0 = 15, moves=[]):
        self.array = array
        self.n = n
        self.m = m
        self.pos0 = pos0
        self.moves = moves
        
    # Переопределение сравнения по стоимости (законченности поля)
    def __lt__(self, other):
        return self.get_total_mdistance() < other.get_total_mdistance()
    def __le__(self, other):
        return self.get_total_mdistance() <= other.get_total_mdistance()
        
    # Для проверки равенства игровых полей, проверяем размерность и костяшки
    def equals(self, board):
        return (self.n == board.n and self.m == board.m and self.array == board.array)
    
    # Для подсчёта костяшек вне позиции
    def get_outofpos_count(self):
        counter = 0
        for i in range(0,len(self.array)):
            # Если проверяем не пустое место и не совпадает позиция
            if (self.array[i] != 0 and i+1 != self.array[i]):
                counter += 1
        return counter
    
    # Manhattan Distance - это расстояние на которое нужно передвинуть
    #костяшку, чтобы она оказалась в позиции. В данном методе мы вычисляем
    #сумму всех расстояний всех костяшек поля.
    def get_total_mdistance(self):
        distance = 0
        for i in range(0,len(self.array)):
            # Если проверяем не пустое место вычисляем расстояние
            elem = self.array[i]
            if elem != 0:
                x = abs((elem-1)%self.n - i%self.n)
                y = abs((elem-1)//self.n - i//self.n)
                distance += x
                distance += y
        return distance
        
    # Для возвращения нового состояния при передвижении костяшки
    def get_moved_state(self, direction):
        # Проверяем направление
        moved_index = None
        if direction == "up":
            moved_index = self.pos0+self.n
        if direction == "down":
            moved_index = self.pos0-self.n
        if direction == "left":
            moved_index = self.pos0+1
        if direction == "right":
            moved_index = self.pos0-1  
        # Если указано правильное направление
        if moved_index != None:
            # Добавление нового хода
            new_moves = self.moves.copy()
            new_moves.append(self.array[moved_index])
            # Замена костяшки и пустого места
            new_array = self.array.copy()
            new_array[self.pos0] = new_array[moved_index]
            new_array[moved_index] = 0
            return BoardState(new_array, self.n, self.m, moved_index, new_moves)
        else:
            print("BoardState.get_moved_state(): Invalid direction, use up/down/left/right\n")
            return 0 
    
def solve_astar(array, n=4, m=4, goal=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0]):
    # Если нерешаемо, возвращаем пустой массив
    if not is_solvable(array):
        return "Not solvable"
    else:
        # Создаём приоритетную очередь, для обработки состояний поля
        pq = queue.PriorityQueue()
        current_board = BoardState(array, n, m, array.index(0))
        # Добавляем в очередь изначальное положение игрового поля
        pq.put(current_board)
        # Два массива для доступа к ещё необработанным и уже обработанным состояниям
        open_boardstates = []
        closed_boardstates = []
        # Пока не найдём нужное поле
        while current_board.array != goal and not pq.empty():
            closed_boardstates.append(current_board)
            # Добавляем в очередь все возможные состояния достижимые из данного
            if current_board.pos0//n != m-1:
                next_board = current_board.get_moved_state("up")
                if not any(x.array == next_board.array for x in closed_boardstates):
                    pq.put(next_board)
            if current_board.pos0//n != 0:
                next_board = current_board.get_moved_state("down")
                if not any(x.array == next_board.array for x in closed_boardstates):
                    pq.put(next_board)
            if current_board.pos0%n != n-1:
                next_board = current_board.get_moved_state("left")
                if not any(x.array == next_board.array for x in closed_boardstates):
                    pq.put(next_board)
            if current_board.pos0%n != 0:
                next_board = current_board.get_moved_state("right")
                if not any(x.array == next_board.array for x in closed_boardstates):
                    pq.put(next_board)
            current_board = pq.get()
        if pq.empty():
            print("Did not found")
        return current_board.moves;
    
# Проверка на работу граничных случаев     
print("moves to solve 1st:", solve_astar([1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0])) #Решаемо
print("moves to solve 2nd:", solve_astar([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0])) #Решаемо
print("moves to solve 3rd:", solve_astar([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14, 0])) #Нерешаемо

# Вывод сложного паззла
start_time = time.time()
moves = solve_astar([9, 5, 14, 15, 13, 6, 7, 11, 12, 10, 8, 2, 3, 4, 1, 0])
print("[solve_astar] moves: ", moves, "\n[solve_astar] total moves:", len(moves)) #Решаемо
print("[solve_astar] took %s seconds" % (time.time() - start_time))

moves to solve 1st: [15, 14, 10, 13, 9, 10, 14, 15]
moves to solve 2nd: []
moves to solve 3rd: Not solvable
[solve_astar] moves:  [2, 8, 1, 4, 3, 12, 13, 9, 5, 14, 15, 11, 8, 2, 4, 3, 12, 13, 9, 5, 14, 15, 11, 8, 2, 4, 3, 12, 10, 1, 12, 3, 4, 12, 7, 2, 8, 11, 15, 6, 2, 15, 6, 2, 1, 7, 15, 6, 2, 1, 7, 10, 3, 15, 6, 7, 5, 14, 1, 2, 11, 8, 12, 4, 15, 3, 10, 6, 3, 15, 4, 12, 8, 11, 7, 3, 6, 10, 13, 9, 14, 5, 10, 14, 9, 13, 14, 10, 3, 7, 11, 8, 7, 11, 2, 3, 10, 6, 11, 7, 8, 2, 3, 10, 6, 11, 12, 4, 15, 12, 11, 6, 10, 3, 2, 8, 4, 11, 12, 15, 11, 12, 6, 10, 3, 2, 8, 4, 7, 8, 2, 3, 10, 6, 8, 7, 12, 8, 6, 10, 3, 2, 7, 3, 10, 6, 8, 12, 3, 7, 4, 3, 12, 8, 6, 10, 7, 6, 8, 12, 3, 4, 6, 3, 12, 8, 3, 7, 2, 6, 7, 3, 8, 12, 3, 8, 12, 3, 8, 7, 6, 2, 7, 6, 4, 8, 3, 12, 6, 3, 8, 4, 3, 7, 10, 6, 15, 11, 12, 8, 7, 10, 6, 15, 10, 7, 8, 12, 11, 14, 15, 10, 12, 11, 14, 12, 11, 14, 12, 15, 10, 11, 14, 12, 15, 14, 11, 10, 14, 15] 
[solve_astar] total moves: 230
[solve_astar] took 1.8047418594360352 seconds
